<a href="https://colab.research.google.com/github/SheikhMudassarHanif/GenAi/blob/main/advanceRagPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setting the APIs

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['LANGCHAIN_API_KEY']=os.getenv('langchain_api')
os.environ["OPENAI_API_KEY"]=os.getenv("openai_api_key")
os.environ["OPENAI_API_BASE"]=os.getenv('openai_api_base')
os.environ["LANGCHAIN_ENDPOINT"]=os.getenv("LANGCHAIN_ENDPOINT")
LANGSMITH_TRACING=True
os.environ["USER_AGENT"] = "LangChainFastAPIApp/1.0"

Loading the data
Transforming the data

In [ ]:
#reading from pd
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.vectorstores import FAISS


from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('attentionPdf.pdf')
docs=loader.load()
#Transformation (text spillter from langchain)
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
# documents[:5]
##Creating Embedding using hugging face

# from langchain_huggingface import HuggingFaceEmbeddings
# embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# db=FAISS.from_documents(documents[:30],embedding_model)

##Creating Embedding using ollama
from langchain_community.embeddings import OllamaEmbeddings
db=FAISS.from_documents(documents[:30],OllamaEmbeddings())
#setting up vector databse

#Now conversion into vectors Vector embedding and vector store



C:\Users\DELL\AppData\Local\Temp\ipykernel_23000\625753536.py:23: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  db=FAISS.from_documents(documents[:30],OllamaEmbeddings())


In [ ]:
db

In [ ]:
query="What is attention is all you need"
result=db.similarity_search(query)
result[0].page_content

'described in section 3.2.\nSelf-attention, sometimes called intra-attention is an attention mechanism relating different positions\nof a single sequence in order to compute a representation of the sequence. Self-attention has been\nused successfully in a variety of tasks including reading comprehension, abstractive summarization,\ntextual entailment and learning task-independent sentence representations [4, 22, 23, 19].\nEnd-to-end memory networks are based on a recurrent attention mechanism instead of sequence-\naligned recurrence and have been shown to perform well on simple-language question answering and\nlanguage modeling tasks [28].\nTo the best of our knowledge, however, the Transformer is the ﬁrst transduction model relying\nentirely on self-attention to compute representations of its input and output without using sequence-\naligned RNNs or convolution. In the following sections, we will describe the Transformer, motivate'

In [ ]:
from langchain_community.llms import Ollama
llm=Ollama(model="llama2")
llm

C:\Users\DELL\AppData\Local\Temp\ipykernel_23000\2415645411.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm=Ollama(model="llama2")


Ollama()

In [ ]:
#Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(
    """ Answer the following question based only on the provided context.
    Think step by step before providing a detailed answer. I will tip you$1000 if the user finds the answer helpful
    <context> {context} </context>
    Question: {input}
    """
)

In [ ]:
##Chain Introduction

from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt)

In [ ]:
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000017775DE50D0>, search_kwargs={})

In [ ]:
##MY retriever chain
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [ ]:
response=retrieval_chain.invoke({"input":"Scaled Dot-Product Attention"})

In [ ]:
response['answer']

'The question is about "Scaled Dot-Product Attention" in the context of the Transformer model.\n\nTo answer this question, we need to understand the role of attention mechanisms in the Transformer architecture. The Transformer model uses self-attention layers in both the encoder and decoder, which allows each position in the sequence to attend to all positions in the sequence. This is different from traditional recurrent neural networks (RNNs) or convolutional neural networks (CNNs), where the attention is limited to a specific window of positions.\n\nScaled dot-product attention is a type of self-attention mechanism used in the Transformer model. It calculates the attention weight between two positions by taking the dot product of their representations and scaling the result by a scalar value. The scalar value, called the attention scale, controls the amount of attention that is given to each position.\n\nThe question asks about preventing leftward information flow in the decoder to p